## Eigenvalue Problem (NGS)

In this notebook we study how to solve an eigenvalue problem using [NGSolve](https://ngsolve.org/) and [SLEPC](https://slepc.upv.es/), in parallel.

__Note__ _to run the code in parallel using a notebook you first need to initialize ipcluster, using the command:_
ipcluster start –engines=MPI -n 4

In [1]:
from ipyparallel import Client
c = Client()
c.ids

[0, 1, 2, 3]

In [2]:
%%px
#STDLIB
from math import pi

#NGSOLVE/NETGEN
from netgen.geom2d import unit_square
from ngsolve import *
from numsa.NGSlepc import *
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
npro = comm.size
H = []
E = []
for k in range(1,2):
    print (rank, npro)
    print("Mesh N. {}".format(k))
    h = 2**(-k);
    E = E + [h]; 
    if comm.rank == 0:
        ngmesh = unit_square.GenerateMesh(maxh=h).Distribute(comm)
    else:
        ngmesh = netgen.meshing.Mesh.Receive(comm)

    mesh = Mesh(ngmesh)

    fes = H1(mesh, order=1, dirichlet=".*")
    u = fes.TrialFunction()
    v = fes.TestFunction()

    a = BilinearForm(fes)
    a += grad(u)*grad(v)*dx
    
    m = BilinearForm(fes)
    m += u*v*dx

    a.Assemble()
    m.Assemble()

    EP = SLEPcEigenProblem("GHEP","krylovschur")
    EP.SpectralTransformation("sinvert")

    PC = EP.KSP.getPC();
    PC.setType("lu");
    PC.setFactorSolverType("mumps");

    EP.setOperators([a.mat,m.mat],fes.FreeDofs())
    EP.setWhich(5);
    
    EP.Solve()
    
    lam, gfur, gfui = EP.getPairs(fes) 
    print([l/pi**2 for l in lam])
    E = E + [abs(lam[0]/pi**2-2)];
    

[stdout:3] Including NGS to PETSC


CompositeError: one or more exceptions from call to method: %px
[Engine Exception]EngineError: Engine 0 died while running task '6d777e3d-092a2adb2353003b04ba54d3_1'
[Engine Exception]EngineError: Engine 1 died while running task '6d777e3d-092a2adb2353003b04ba54d3_2'
[Engine Exception]EngineError: Engine 2 died while running task '6d777e3d-092a2adb2353003b04ba54d3_3'
[Engine Exception]EngineError: Engine 3 died while running task '6d777e3d-092a2adb2353003b04ba54d3_4'

In [ ]:
E

In [ ]:
%%px
#STDLIB
from math import pi

#NGSOLVE/NETGEN
from netgen.geom2d import unit_square
from ngsolve import *
import ngsolve.ngs2petsc as N2P


#MPI
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
npro = comm.size
if rank == 0:
    print (rank, npro)

if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.1).Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)

mesh = Mesh(ngmesh)

fes = H1(mesh, order=1, dirichlet=".*")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
pre = Preconditioner(a, type="direct", inverse="masterinverse")

m = BilinearForm(fes)
m += u*v*dx

a.Assemble()
m.Assemble()

EP = N2P.SLEPcEigenProblem("GHEP","krylovschur")
EP.SpectralTransformation("sinvert")

PC = EP.KSP.getPC();
PC.setType("lu");
PC.setFactorSolverType("mumps");

EP.setOperators([a.mat,m.mat],fes.FreeDofs())
EP.setWhich(5);

EP.Solve()

lam, gfur, gfui = EP.getPairs(fes) 
if rank == 0:
    print([l/pi**2 for l in lam])
eig1 = GridFunction(fes); eig1.vec.data = gfur.vecs[0].data; 
eig2 = GridFunction(fes); eig2.vec.data = gfur.vecs[1].data;
eig3 = GridFunction(fes); eig3.vec.data = gfur.vecs[2].data; 
eig4 = GridFunction(fes); eig4.vec.data = gfur.vecs[3].data;

In [ ]:
eig1 = c[:]["eig1"]
eig2 = c[:]["eig2"]
eig3 = c[:]["eig3"]
eig4 = c[:]["eig4"]
Draw (eig1[0])
Draw (eig2[0])
Draw (eig3[0])
Draw (eig4[0])